In [3]:
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
import numpy as np

In [4]:
stop_times = pd.read_csv('dados\paradas\stop_times.txt', sep=',')
stop_times['arrival_time'] = pd.to_datetime(stop_times['arrival_time'])
stop_times['hour'] = stop_times['arrival_time'].dt.hour

In [5]:
stops = pd.read_csv('dados\paradas\stops.txt', sep=',')

In [6]:
events = pd.merge(stop_times, stops, on='stop_id')
print(events.count())

trip_id           95731
arrival_time      95731
departure_time    95731
stop_id           95731
stop_sequence     95731
hour              95731
stop_name         95731
stop_desc         92027
stop_lat          95731
stop_lon          95731
dtype: int64


In [ ]:
av_reboucas1 = events[events['stop_name'].str.contains("Av. Paulista", case=False, na=False)]
print(av_reboucas1.describe())
av_reboucas2 = events[events['stop_desc'].str.contains("Av. Paulista", case=False, na=False)]
print(av_reboucas2.describe())

av_reboucas = pd.concat([av_reboucas1,av_reboucas2],ignore_index=True).drop_duplicates()
av_reboucas = av_reboucas.groupby('stop_id', as_index=False).count()
print(av_reboucas.describe())

In [8]:
stp = [630015016, 630015011, 6311298, 6311297, 260016861, 260016860]

av_reboucas = events[events['stop_id'].isin(stp)]
print(av_reboucas.describe)


<bound method NDFrame.describe of          trip_id        arrival_time departure_time    stop_id  stop_sequence  \
23128  178L-10-0 2023-02-24 18:14:00       18:14:00  260016860             38   
23129  669A-10-1 2023-02-24 16:24:00       16:24:00  260016860             11   
23130  701A-10-0 2023-02-24 12:40:14       12:40:14  260016860             63   
23131  701U-10-0 2023-02-24 08:25:45       08:25:45  260016860             16   
23132  702C-10-1 2023-02-24 11:30:53       11:30:53  260016860             18   
...          ...                 ...            ...        ...            ...   
85786  715M-10-1 2023-02-24 16:40:15       16:40:15  630015011             22   
85787  775P-10-1 2023-02-24 17:29:30       17:29:30  630015011             16   
85788  809V-10-0 2023-02-24 16:56:30       16:56:30  630015011             31   
85789  857P-10-1 2023-02-24 17:28:48       17:28:48  630015011             13   
85790  857R-10-1 2023-02-24 17:55:48       17:55:48  630015011             

In [9]:
#filter_es_morning = av_reboucas.loc[(av_reboucas['hour'] >= 7) & (av_reboucas['hour'] < 9)].groupby('stop_id', as_index=False).count()
filter_es_morning = av_reboucas.groupby('stop_id', as_index=False).count()
filter_es_morning = filter_es_morning[['stop_id', 'trip_id']]
filter_es_morning = filter_es_morning.rename(columns={'trip_id': 'qtd'})


filter_es_morning= pd.merge(filter_es_morning, stops, on='stop_id')
print(filter_es_morning)


     stop_id  qtd                  stop_name  \
0    6311297   14  Parada 1 - Faria Lima B/C   
1    6311298   12  Parada 1 - Faria Lima C/B   
2  260016860   24             Caio Prado C/B   
3  260016861   26             Caio Prado B/C   
4  630015011    5  Parada 2 - Faria Lima C/B   
5  630015016    6  Parada 2 - Faria Lima B/C   

                                           stop_desc   stop_lat   stop_lon  
0  Av. Eusébio Matoso Ref.: Cruzamento Av. Eusébi... -23.571165 -46.691091  
1  Av. Rebouças, 3215 Ref.: Cruzamento Av. Rebouç... -23.570806 -46.690569  
2  R. Da Consolação, 896 Ref.: R Marques De Paran... -23.548312 -46.650406  
3  R. Da Consolação, 901 Ref.: R Marques De Paran... -23.548174 -46.649565  
4  Av. Rebouças, 3215 Ref.: Cruzamento Av. Rebouç... -23.570905 -46.690499  
5  Av. Eusébio Matoso, 133 Ref.: Cruzamento Av. E... -23.571182 -46.691378  


In [13]:
fig = go.Figure()

fig = px.scatter_mapbox(filter_es_morning, lat="stop_lat", lon="stop_lon", zoom=9, width=600, height=500)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

def SetColor(df):
    values = df['qtd'].tolist()
    color_list = []
    for i in values:
        if(i >= 10):
            color_list.append("red")
        else:
            color_list.append("blue")
    
    return color_list

def SetSize(df):
    values = df['qtd'].tolist()
    color_list = []
    for i in values:
        if(i >= 10):
            color_list.append(10)
        else:
            color_list.append(4)
    
    return color_list

fig.add_trace(go.Scattermapbox(
        lat=filter_es_morning['stop_lat'].tolist(),
        lon=filter_es_morning['stop_lon'].tolist(),
        mode='markers',
        marker=go.scattermapbox.Marker(
            size= SetSize(filter_es_morning),
            color= SetColor(filter_es_morning)  #function gets called here and will return a list of colors, (i.e. ['green', 'blue', 'red', 'green'])
        ),
    )
)

fig.show()